In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer
import string
import nltk
nltk.download('stopwords')
import os
import fnmatch
import io
from pprint import pprint
from time import time  # to time our operations

#LDA
import warnings
warnings.filterwarnings('ignore')
import os 
%matplotlib inline
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess, tokenize
from gensim.models import CoherenceModel

def stopwords_city(City):
    month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december' ]
    day = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    if City == 'Boston':
        return month + day + ['boston', 'roxbury','fiandaca','opencounter', 'accela','forth', 'ortega', 'burro','theroxburyinnovation', 'julie', 'elliott', '21stcentury', 'nstar','dorsey','austin', 'blackmon', 'swett', 'ofboston', 'paired','ass','first', 'martin', 'addition', 'every', 'walsh', 'according', 'please visit', 'noted',
            'please','april', 'february', 'visit', 'across', 'december', 'https', 'following',
             'noted', 'along', 'three', 'total', 'pointed', 'citys', 'announced', 'massachusetts', 'million']
    if City =='Taipei':
        return month + day +['taipei','since', 'dadaocheng', 'serf', 'addition', 'february','related', 'october', 'january', 'around', 'taking', 'march', 'first', 'according', 'please visit',
                'noted', 'please','april', 'among''february', 'visit', 'across', 'december', 'https', 'ko',
                'following', 'universiade','september','november','august','noted', 'along', 'three', 'total', 'pointed', 'citys', 'chinese']
    if City == 'Seoul':
        return month + day +['seoul','soon','yeomchang', 'dulle','three', 'gaehwasinnonhyeon', 'major', 'first', 'million', 'various','selected','around','number', 'based','total',
                'order','special', 'since']
    if City == 'Helsinki':
        return month + day + ['helsinki','based', '6aika', 'wrtsil','january', 'first', 'sitra']
    else:
        return month + day
    
def corpus(City):

    folder = '/Users/clementnicolas/JournalPaper3/{}_News'.format(City)

    corpus=[]
    tokenizer = RegexpTokenizer(r'\w+')
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords.extend(stopwords_city(City))
    with_stopwords=0
    without_stopwords=0
    articles=0
    tokens =0
    for file in os.listdir(folder):
        
        path = os.path.join(folder, file)
        if fnmatch.fnmatch(file, '*.txt'):
            with io.open(path, 'r', encoding="UTF-8") as fin:
                content = fin.read()
                data = content.splitlines()
                #articles=articles+len(data)


            for sentence in data:
                word_list = tokenizer.tokenize(sentence.lower())
                word_list1 = [word for word in word_list if word.isalpha()]
                word_list2 = [word for word in word_list1 if len(word)>3]
                word_list3 = [word for word in word_list2 if word not in stopwords]
                if len(word_list3)>20:
                    corpus.append(word_list3)
                    #with_stopwords=with_stopwords+len(word_list2)
                    #without_stopwords = without_stopwords+len(word_list3)
                    #tokens = tokens+len(word_list)
    
    
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(corpus, min_count=5, threshold=100)  # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[corpus], threshold=100)

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]

    ######## For BIGRAMS
    corpus2 = make_bigrams(corpus)
    
    ######## For TRIGRAMS
    
    #corpus22 = make_bigrams(corpus)
    #corpus2 = make_trigrams(corpus22)

    #print(corpus2[:10])
    L=[]
    lemmatizer = WordNetLemmatizer()
    for w in range(len(corpus2)):
        x = 0
        while x <(len(corpus2[w])):
            #if corpus2[w][x] not in L:
            #    L.append(corpus2[w][x])
            corpus2[w][x] = lemmatizer.lemmatize(corpus2[w][x])
            if corpus2[w][x] in stopwords:
                del corpus2[w][x]
            x += 1
                
    return corpus2